# Run this code first

In [6]:
import pandas as pd
import numpy as np
from chainer import serializers
from chainer import Link, Chain, ChainList
import chainer.links as L
import chainer.functions as F

##################################################################################
##################################################################################
class MyNetwork1(Chain):

    def __init__(self, n_mid_units=100, n_out=794):
        super(MyNetwork1, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_mid_units)
            #self.l2 = L.Linear(n_mid_units, n_out)
            self.l2 = L.Linear(n_mid_units, n_mid_units)
            self.l3 = L.Linear(n_mid_units, n_out)

    def __call__(self, x):
        h = F.relu(self.l1(x))
        #return self.l2(h)
        h = F.relu(self.l2(h))
        return self.l3(h)
    
class MyNetwork2(Chain):

    def __init__(self, n_mid_units=100, n_out=986):
        super(MyNetwork2, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_mid_units)
            #self.l2 = L.Linear(n_mid_units, n_out)
            self.l2 = L.Linear(n_mid_units, n_mid_units)
            self.l3 = L.Linear(n_mid_units, n_out)

    def __call__(self, x):
        h = F.relu(self.l1(x))
        #return self.l2(h)
        h = F.relu(self.l2(h))
        return self.l3(h)


##################################################################################
##################################################################################

infer_model_0 = MyNetwork1()
serializers.load_npz('first_half', infer_model_0)

infer_model_1 = MyNetwork2()
serializers.load_npz('second_half', infer_model_1)

#################################################################################
#################################################################################



C:\Users\bruce\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Then, define function bellow

In [7]:
def returns(x):
    average = [15.5305463609422,80.66675753041741,0.0926605082089513,0.21238504591770846,
            128.33909339905523,152.0165825135359,68.93027805310743,50.079400025728795,
            0.3778091024517165,66.76580619022782]
    std = [2.3005193204939567,5.639387429970276,0.017966913673662474,0.05710810082312886,
            24.685619804620845,34.64006633570716,8.278365359007248,18.359870840871817,
            0.3210542977240008,20.59842003865487]

    x = (pd.DataFrame(x) - pd.DataFrame(average))/pd.DataFrame(std)
    x = x.as_matrix()
    x = x.flatten()
    x = x.flatten()
    x = np.ndarray.astype(x, "float32")
    
    x = x[None, ...]
    
    y0 = infer_model_0(x)
    y1 = infer_model_1(x)
    
    y0 = y0.data
    pred_label0 = y0.argmax(axis=1)
    pred_label0= (int(pred_label0)-1)/1000 - 0.559
    
    y1 = y1.data
    pred_label1 = y1.argmax(axis=1)
    pred_label1 = (int(pred_label1)-1)/1000 - 0.745
    
    return (pred_label0, pred_label1)

### This is just sample X values


In [8]:
x = [14.983558,76.256631,0.058866,0.231032,106.599460,113.357185,60.364280,74.864088,0.267517,64.210913]

In [9]:
ea = returns(x)[0]
eb = returns(x)[1]

print(ea,eb)

-0.14400000000000007 0.02400000000000002


### Should work now

In [10]:
ticker_df = pd.read_csv('stock_data/ticker_data.csv') ## Please change the directory where you save stock_data file.
factor_df = pd.read_csv('stock_data/factor_data.csv') ## Same as above.
ticker_df.set_index('timestep', inplace=True)
factor_df.set_index('timestep', inplace=True)
stock_df = ticker_df.join(factor_df, how='left')

In [11]:
for t in range(0, 500):   
    if t == 0:
        first_rt = stock_df[stock_df['ticker']==t]['returns']
    else:
        first_rt = first_rt + stock_df[stock_df['ticker']==t]['returns']
        
for t in range(500, 1000):   
    if t == 500:
        second_rt = stock_df[stock_df['ticker']==t]['returns']
    else:
        second_rt = second_rt + stock_df[stock_df['ticker']==t]['returns']

## Error calculation

In [12]:
subdata = stock_df[stock_df['ticker']==0]

In [13]:
X_training = subdata[['VIX', 'COPP', "3M_R", "US_TRY", "BIG_IX","SMALL_IX", "SENTI", "TEMP", "RAIN", "OIL"]]
X_training = X_training.as_matrix(columns=None)

In [14]:
error = []
for ind in range(1,2519):
    x = list(X_training[ind])
    prdy = returns(x)[0]
    y = first_rt[ind+1]
    error.append(prdy/y)
    #print(rt)
    #print(first_rt[ind+1])
    #print(second_rt[ind+1])

In [18]:
error

[1.0005382943269427,
 1.0020727505394897,
 1.0091612421876972,
 1.0079275358548783,
 0.9946194611390092,
 0.9986137386146965,
 0.9826597676501732,
 0.9979212049578682,
 0.9994103518586767,
 0.9994907034990204,
 1.000068676038867,
 0.9784568645343308,
 0.9974197079746808,
 0.9951303621872648,
 0.9871944934845649,
 1.0088272423369564,
 1.0077795109783592,
 1.0135077662381138,
 1.003217425630502,
 0.9973910067439941,
 1.0013864495934868,
 0.9941934473418356,
 1.0243713537473043,
 1.001659771301852,
 1.0060426567458658,
 1.0033604378044332,
 0.9518751108155327,
 1.010947644870972,
 1.0018542461704085,
 0.9967966980657043,
 0.9993298769071304,
 0.9915017343307571,
 0.9870802381534765,
 1.000398538289308,
 1.048701036496759,
 0.9639530709811198,
 0.997715082143346,
 0.9496937132924577,
 0.9979348823985811,
 0.9915170846218916,
 1.147935989820464,
 0.9865234843244052,
 0.6819864103389479,
 0.9940946613981322,
 0.9987308280557183,
 0.9918843150073888,
 1.004284777751232,
 1.0020775611294983,
 

In [29]:
sum(error)/len(error)

0.9949162102105191

In [21]:
error = []
for ind in range(1,2519):
    x = list(X_training[ind])
    prdy = returns(x)[1]
    y = second_rt[ind+1]
    error.append(prdy/y)
    #print(rt)
    #print(first_rt[ind+1])
    #print(second_rt[ind+1])

In [22]:
sum(error)/len(error)

0.9951035536852163

In [36]:
MAX_LOOKBACK = 50

class PortfolioGenerator(object):

    def __init__(self):
        pass

    def read_stock_data(self):
        '''
        Description:
            Reads in simulated stock data from stock_data.csv
        Returns:
            stock_df (DataFrame): standardized ticker/factor data in pandas df
        Raises:
            AssertionError: ticker_data.csv/factor_data.csv has an invalid format
        '''
        ticker_df = pd.read_csv('stock_data/ticker_data.csv')
        factor_df = pd.read_csv('stock_data/factor_data.csv')
        assert 'timestep' in ticker_df.columns, "ticker_data.csv has an invalid format"
        assert 'ticker' in ticker_df.columns, "ticker_data.csv has an invalid format"
        assert 'returns' in ticker_df.columns, "ticker_data.csv has an invalid format"
        assert 'timestep' in factor_df.columns, "factor_data.csv has an invalid format"
        ticker_df.set_index('timestep', inplace=True)
        factor_df.set_index('timestep', inplace=True)
        stock_df = ticker_df.join(factor_df, how='left')
        return stock_df

    def build_signal(self, stock_features):
        '''
        Description:
            Using stock features, generate a buy/sell stock
            signal (suggested from -100 to 100 where -100 is strong
            sell signal) for the stock on a given day.
        Args:
            hist_features (DataFrame) - dataframe of historical features
        Return:
            signal (pandas Series) - key = ticker, value = signal
        Raises:
            NotImplementedError - throws error if the function is not implemented
        '''
        raise NotImplementedError("build_signal must be implemented")

    def simulate_portfolio(self):
        '''
        Description:
            Simulates performance of the portfolio on historical data
        Return:
            sharpe (int) - sharpe ratio for the portfolio
        '''
        daily_returns = []
        stock_df = self.read_stock_data()
        for idx in stock_df.index.unique():
            print("timestep", idx)
            if idx < MAX_LOOKBACK:
                continue
            stock_features = stock_df.loc[idx-MAX_LOOKBACK:idx-1]
            returns = stock_df.loc[idx:idx].set_index('ticker')['returns']
            signal = self.build_signal(stock_features)
            signal_return = returns *signal
            daily_returns.append(np.mean(signal_return))
            #print(daily_returns[idx-50])
        sharpe_ratio = np.sqrt(252) * (np.mean(daily_returns) / np.std(daily_returns))
        #pd.DataFrame(daily_returns,columns=['a']).to_csv('dailyreturn_market.csv')
        
        return sharpe_ratio

In [138]:
class SampleStrategy(PortfolioGenerator):

    def __init__(self):
        pass

    def build_signal(self, stock_features):
        return self.momentum(stock_features)

    def momentum(self, stock_features):
        #i=stock_features['returns'].index.values[-1]+1
        signals=list(stock_features[['VIX','COPP','3M_R','US_TRY','BIG_IX','SMALL_IX','SENTI','TEMP','RAIN','OIL']].iloc[-1])
        my_predict=returns(signals)
        #print(my_predict)
        ea=my_predict[0]
        eb=my_predict[1]
        #print(ea)
        if abs(ea-eb)>1e-10:
            lambda1=(0.11-eb)/(ea-eb)
        else:
            lambda1=1/2
            #print("**************************************************************")
        my_weights=np.zeros(1000)
        my_weights[:500]=lambda1
        my_weights[500:]=(1-lambda1)
        #print(lambda1)
        #print(my_weights)
        return my_weights
# Test out performance by running 'python sample_strategy.py'
if __name__ == "__main__":
    portfolio = SampleStrategy()
    sharpe = portfolio.simulate_portfolio()
    print("*** Strategy Sharpe is {} ***".format(sharpe))

timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 5
timestep 6
timestep 7
timestep 8
timestep 9
timestep 10
timestep 11
timestep 12
timestep 13
timestep 14
timestep 15
timestep 16
timestep 17
timestep 18
timestep 19
timestep 20
timestep 21
timestep 22
timestep 23
timestep 24
timestep 25
timestep 26
timestep 27
timestep 28
timestep 29
timestep 30
timestep 31
timestep 32
timestep 33
timestep 34
timestep 35
timestep 36
timestep 37
timestep 38
timestep 39
timestep 40
timestep 41
timestep 42
timestep 43
timestep 44
timestep 45
timestep 46
timestep 47
timestep 48
timestep 49
timestep 50
timestep 51
timestep 52
timestep 53
timestep 54
timestep 55
timestep 56
timestep 57
timestep 58
timestep 59
timestep 60
timestep 61
timestep 62
timestep 63
timestep 64
timestep 65
timestep 66
timestep 67
timestep 68
timestep 69
timestep 70
timestep 71
timestep 72
timestep 73
timestep 74
timestep 75
timestep 76
timestep 77
timestep 78
timestep 79
timestep 80
timestep 81
timestep 82
timestep 83
ti

timestep 644
timestep 645
timestep 646
timestep 647
timestep 648
timestep 649
timestep 650
timestep 651
timestep 652
timestep 653
timestep 654
timestep 655
timestep 656
timestep 657
timestep 658
timestep 659
timestep 660
timestep 661
timestep 662
timestep 663
timestep 664
timestep 665
timestep 666
timestep 667
timestep 668
timestep 669
timestep 670
timestep 671
timestep 672
timestep 673
timestep 674
timestep 675
timestep 676
timestep 677
timestep 678
timestep 679
timestep 680
timestep 681
timestep 682
timestep 683
timestep 684
timestep 685
timestep 686
timestep 687
timestep 688
timestep 689
timestep 690
timestep 691
timestep 692
timestep 693
timestep 694
timestep 695
timestep 696
timestep 697
timestep 698
timestep 699
timestep 700
timestep 701
timestep 702
timestep 703
timestep 704
timestep 705
timestep 706
timestep 707
timestep 708
timestep 709
timestep 710
timestep 711
timestep 712
timestep 713
timestep 714
timestep 715
timestep 716
timestep 717
timestep 718
timestep 719
timestep 720

timestep 1262
timestep 1263
timestep 1264
timestep 1265
timestep 1266
timestep 1267
timestep 1268
timestep 1269
timestep 1270
timestep 1271
timestep 1272
timestep 1273
timestep 1274
timestep 1275
timestep 1276
timestep 1277
timestep 1278
timestep 1279
timestep 1280
timestep 1281
timestep 1282
timestep 1283
timestep 1284
timestep 1285
timestep 1286
timestep 1287
timestep 1288
timestep 1289
timestep 1290
timestep 1291
timestep 1292
timestep 1293
timestep 1294
timestep 1295
timestep 1296
timestep 1297
timestep 1298
timestep 1299
timestep 1300
timestep 1301
timestep 1302
timestep 1303
timestep 1304
timestep 1305
timestep 1306
timestep 1307
timestep 1308
timestep 1309
timestep 1310
timestep 1311
timestep 1312
timestep 1313
timestep 1314
timestep 1315
timestep 1316
timestep 1317
timestep 1318
timestep 1319
timestep 1320
timestep 1321
timestep 1322
timestep 1323
timestep 1324
timestep 1325
timestep 1326
timestep 1327
timestep 1328
timestep 1329
timestep 1330
timestep 1331
timestep 1332
timest

timestep 1850
timestep 1851
timestep 1852
timestep 1853
timestep 1854
timestep 1855
timestep 1856
timestep 1857
timestep 1858
timestep 1859
timestep 1860
timestep 1861
timestep 1862
timestep 1863
timestep 1864
timestep 1865
timestep 1866
timestep 1867
timestep 1868
timestep 1869
timestep 1870
timestep 1871
timestep 1872
timestep 1873
timestep 1874
timestep 1875
timestep 1876
timestep 1877
timestep 1878
timestep 1879
timestep 1880
timestep 1881
timestep 1882
timestep 1883
timestep 1884
timestep 1885
timestep 1886
timestep 1887
timestep 1888
timestep 1889
timestep 1890
timestep 1891
timestep 1892
timestep 1893
timestep 1894
timestep 1895
timestep 1896
timestep 1897
timestep 1898
timestep 1899
timestep 1900
timestep 1901
timestep 1902
timestep 1903
timestep 1904
timestep 1905
timestep 1906
timestep 1907
timestep 1908
timestep 1909
timestep 1910
timestep 1911
timestep 1912
timestep 1913
timestep 1914
timestep 1915
timestep 1916
timestep 1917
timestep 1918
timestep 1919
timestep 1920
timest

timestep 2438
timestep 2439
timestep 2440
timestep 2441
timestep 2442
timestep 2443
timestep 2444
timestep 2445
timestep 2446
timestep 2447
timestep 2448
timestep 2449
timestep 2450
timestep 2451
timestep 2452
timestep 2453
timestep 2454
timestep 2455
timestep 2456
timestep 2457
timestep 2458
timestep 2459
timestep 2460
timestep 2461
timestep 2462
timestep 2463
timestep 2464
timestep 2465
timestep 2466
timestep 2467
timestep 2468
timestep 2469
timestep 2470
timestep 2471
timestep 2472
timestep 2473
timestep 2474
timestep 2475
timestep 2476
timestep 2477
timestep 2478
timestep 2479
timestep 2480
timestep 2481
timestep 2482
timestep 2483
timestep 2484
timestep 2485
timestep 2486
timestep 2487
timestep 2488
timestep 2489
timestep 2490
timestep 2491
timestep 2492
timestep 2493
timestep 2494
timestep 2495
timestep 2496
timestep 2497
timestep 2498
timestep 2499
timestep 2500
timestep 2501
timestep 2502
timestep 2503
timestep 2504
timestep 2505
timestep 2506
timestep 2507
timestep 2508
timest

In [129]:
l=[]
for i in range(50,300):
    my_predict=returns(list(X_training[i]))
    #print(my_predict)
    ea=my_predict[0]
    eb=my_predict[1]
    #print(ea)
    if abs(ea-eb)>1e-10:
        lambda1=(5e-2-eb)/(ea-eb)
    else:
        lambda1=1/2
    l.append(lambda1*first_rt[i+1]+(1-lambda1)*second_rt[i+1])

In [130]:
sqrt(252)*pd.DataFrame(l).mean()/pd.DataFrame(l).std()

0    275.541737
dtype: float64

In [139]:
first_rt.mean()

0.002701518038597828

In [101]:
pd.DataFrame(l).mean()

0    0.995199
dtype: float64

In [112]:
pd.DataFrame(l).std()

0    0.001907
dtype: float64

In [116]:
from math import sqrt

In [140]:
def predict_num(x):
    return np.random.normal(x,0.05*x)
def comb2(mylist,lamda1):
    return (lamda1*(mylist[1]+mylist[0])+(1-lamda1)*(mylist[2]+mylist[3]))
def comb4(mylist,lamda1,lamda2,lamda3):
    return (lamda1*mylist[1]+lamda2*mylist[0]+lamda3*mylist[2]+(1-lamda1-lamda2-lamda3)*mylist[3])

In [159]:
discrepancy2=[]
discrepancy4=[]
for j in range(0,10000,1):
        predlist=[]
        truelist=[]
        for i in range(0,4,1):
            predlist+=[predict_num(10+0.2*i)]
            truelist+=[10+0.2*i]
        #lamda1=np.random.uniform(0,1)
        lamda2=np.random.uniform(0,1)
        lamda3=np.random.uniform(0,1-lamda2)
        lamda4=np.random.uniform(0,1-lamda2-lamda3)
        lamda1=0.5
        lamda2=0.25
        lamda3=0.25
        lamda4=0.25
        discrepancy2+=[abs(comb2(predlist,lamda1)-comb2(truelist,lamda1))]
        discrepancy4+=[abs(comb4(predlist,lamda2,lamda3,lamda4)-comb4(truelist,lamda2,lamda3,lamda4))]
print (sum(discrepancy2)/len(discrepancy2))
print (sum(discrepancy4)/len(discrepancy4))

0.4073837098229452
0.20369185491147254
